### Required Setup

If you don't have Python on your computer, you can use the [Anaconda Python distribution](http://continuum.io/downloads) to install most of the Python packages you need. Anaconda provides a simple double-click installer for your convenience.

This notebook uses several Python packages that come standard with the Anaconda Python distribution. The primary libraries that we'll be using are:

* **NumPy**: Provides a fast numerical array structure and helper functions.
* **pandas**: Provides a DataFrame structure to store data in memory and work with it easily and efficiently.
* **scikit-learn/sklearn**: The essential Machine Learning package in Python.
* **matplotlib**: Basic plotting library in Python; most other Python plotting libraries are built on top of it.
* **Seaborn**: Advanced statistical plotting library.
* **waterqmark**: A Jupyter Notebook extension for printing timestamps, version numbers, and hardware information.

To make sure you have all of the packages you need, install them with `conda`:

```
conda create -n SYSC4415_tutorials python=3.11
conda activate SYSC4415_tutorials

conda install jupyter
conda install numpy pandas scikit-learn matplotlib seaborn graphviz statsmodels
conda install -c conda-forge watermark

```

`conda` may ask you to update some of them if you don't have the most recent version. Allow it to do so.

**Note:** I will not support people trying to run this notebook outside of the Anaconda Python distribution.


### _New Stuff to Install for this Tutorial_:

```
conda install sympy

```


# Tutorial 4 - Gradient Descent

**Course:** SYSC 4415 - Introduction to Machine Learning

**Semester:** Winter 2025

**Adapted by:** [Kevin Dick](https://kevindick.ai/), [Igor Bogdanov](igorbogdanov@cmail.carleton.ca) 

**Adapted From:** [Daniel Newman](https://github.com/dtnewman)'s [Notebook](https://nbviewer.jupyter.org/github/dtnewman/gradient_descent/blob/master/stochastic_gradient_descent.ipynb)

and [Gradient Descent Notebook](https://github.com/udlbook/udlbook/blob/main/Notebooks/Chap06/6_2_Gradient_Descent.ipynb) from [Understanding Deep Learning](https://udlbook.github.io/udlbook/) by [Simon J.D. Prince](http://udlbook.com)

---

In this notebook we will:
1. define gradient descent (GD)
2. visualize GD applied to an arbitrary function
3. use GD to fit a linear regression model

## 1. Defining Gradient Descent

**Gradient descent**, also known as <b>steepest descent</b>, is an optimization algorithm for finding the local minimum of a function. 

**Assumption:** the function must be differentiable.

To find a local minimum, the function _"steps"_ in the  direction of the negative of the gradient. <b>Gradient ascent</b> is the same as gradient descent, except that it steps in the direction of the positive of the gradient and therefore finds local maximums instead of minimums. 

The algorithm of gradient descent can be outlined as follows:

&nbsp;&nbsp;&nbsp; 1: &nbsp; Choose initial guess $x_0$ <br>
&nbsp;&nbsp;&nbsp;    2: &nbsp; <b>for</b> k = 0, 1, 2, ... <b>do</b> <br>
&nbsp;&nbsp;&nbsp;    3:   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $s_k$ = -$\nabla f(x_k)$ <br>
&nbsp;&nbsp;&nbsp;    4:   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $x_{k+1} = x_k + \alpha_k s_k$ <br>
&nbsp;&nbsp;&nbsp;    5: &nbsp;  <b>end for</b>


## 2. Visualizing Gradient Descent

As a simple example, let's find a local minimum for the function $f(x) = x^3-2x^2+2$

In [ ]:
%matplotlib inline
from sympy import *
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import random
from scipy import stats
from scipy.optimize import fmin
from IPython.display import clear_output
from time import sleep

def plot_curve(x_list, y_list):
  """ plot_curve
      Helper function to iteratively plot
      arbitrary curves.
  """
  titles = ["Gradient descent", 
            "Gradient descent (zoomed in)", 
            "Gradient descent (zoomed most)"]
  xlims = [[-1,2.5],
           [1.2,2.1],
           [1.32,1.34]]
  plt.figure(figsize=[10,3])
  for i in range(3):
    plt.subplot(1,3,i+1)
    plt.scatter(x_list,y_list,c="r")
    plt.plot(x_list,y_list,c="r")
    plt.plot(x,f(x), c="b")
    plt.xlim(xlims[i])
    plt.ylim([0,3])
    plt.title(titles[i])
  plt.show()

def plot_crickets(data, theta_new=None):
  """ plot_curve
      Helper function to iteratively plot
      arbitrary curves.
      ---
      theta_new, if defined, is a two-tuple of estimated parameters.
  """
  xx = np.linspace(0,21,1000)
  plt.scatter(data[:, 0], data[:, 1], marker='o', c='b')
  if not theta_new is None: plt.plot(xx,h(theta_new[0],theta_new[1],xx))
  plt.title('Striped Ground Cricket Chirp Frequency vs. Temperature')
  plt.xlabel('Frequency (chirps/sec)')
  plt.ylabel('Temperature (F)')
  plt.xlim([13,21])
  plt.ylim([65,95])
  plt.show()

def plot_generic(data, theta_new=None):
  """ plot_generic
      Helper function to iteratively plot
      arbitrary curves.
      ---
      theta_new, if defined, is a two-tuple of estimated parameters.
  """
  xx = np.linspace(0,100,1000)
  plt.scatter(data[:, 0], data[:, 1], marker='o', c='b', alpha=0.1)
  if not theta_new is None: plt.plot(xx,h(theta_new[0],theta_new[1],xx), c="r", zorder=999)
  plt.xlabel('x0')
  plt.ylabel('x1')
  plt.title(f'num. points: {len(data[:, 0])}')
  plt.show() 

In [ ]:
# Explicitly: We can create our function and derivative explicitly with lambdas
# our function
f = lambda x: x**3-2*x**2+2

# the derivative of our function
f_prime = lambda x: 3*x**2-4*x

In [ ]:
# Alternatively: We can use the SymPy library to do this for us
# We have to create a "symbol" called x
X = Symbol('x')
f = X**3-2*X**2+2
f_prime = f.diff(X)

# We can then automatically turn them into labmda functions
f = lambdify(X, f)
f_prime = lambdify(X, f_prime)

In [ ]:
# Plot the function
x = np.linspace(-1,2.5,1000)
plt.plot(x,f(x))
plt.xlim([-1,2.5])
plt.ylim([0,3])
plt.ylabel('f(x)')
plt.xlabel('x')
plt.show()

We can see from plot above that our local minimum is gonna be near around 1.4 or 1.5 (on the x-axis), but let's pretend that we don't know that, so we set our starting point (arbitrarily, in this case) at $x_0 = 2$

In [ ]:
# Setup some variables
x_old     = 0       # Initialized for plotting
x_new     = 2       # The algorithm starts at x = 2
n_k       = 0.1     # step size
precision = 0.0001  # threshold to stop iterations
delay     = 1       # one second of sleep to visualiize

# Lists to track values to plot
x_list, y_list = [x_new], [f(x_new)]
 
while abs(x_new - x_old) > precision:
  # Display output each iteration
  clear_output(wait=True)
  print(f'Iteration {len(x_list)}\tx: {x_new}\tdiff: {abs(x_new - x_old)}')
  plot_curve(x_list, y_list)
  sleep(delay) # To visualize

  # Apply Gradient Descent
  x_old = x_new
  s_k = -f_prime(x_old)
  x_new = x_old + n_k * s_k
  x_list.append(x_new)
  y_list.append(f(x_new))

# Print the x-value at the local minimum
print('Local minimum occurs at: {:.3f}'.format(x_new))
print("Number of steps:", len(x_list))

### Constant Step Size & [Google's Toy Example](https://developers.google.com/machine-learning/crash-course/fitter/graph)
You'll notice that the step size (also called **learning rate**) in the implementation above is **constant**. Doing this makes it easier to implement the algorithm. 

However, it also presents some issues: If the **step size is too small**, then convergence will be **very slow**, but if we make it **too large**, then the method may **fail to converge at all**. 

### Time-Based Learning Rate Decay Schedule
A solution to this is to update the step size is choosing a decrease constant $d$ that shrinks the step size over time (**Time-based**):
$\eta(t+1) = \frac{\eta(t)}{(1+t \times d)}$.

Other types of [learning rate schedules](https://en.wikipedia.org/wiki/Learning_rate#Learning_rate_schedule) can also be used, including:
* **Step-based** learning schedules changes the learning rate according to some pre-defined steps.
* **Exponential** learning schedules are similar to step-based but instead of steps a decreasing exponential function is used.

### (Advanced Reading): ["Why Momentum Really Works"](https://distill.pub/2017/momentum/)

In [ ]:
# Setup the same variables
x_old     = 0       # Initialized for plotting
x_new     = 2       # The algorithm starts at x = 2
n_k       = 0.17    # step size (starting at 0.17 to show effect)
precision = 0.0001  # threshold to stop iterations
delay     = 1       # one second of sleep to visualiize
t,d       = 0,1     # NEW Time-tracking variables and Decay-value

# Lists to track values to plot
x_list, y_list = [x_new], [f(x_new)]
 
while abs(x_new - x_old) > precision:
  # Display output each iteration
  clear_output(wait=True)
  print(f'Iteration {len(x_list)}\tx: {x_new}\tdiff: {abs(x_new - x_old)}')
  plot_curve(x_list, y_list)
  sleep(delay) # To visualize
  
  
  x_old = x_new
  s_k = -f_prime(x_old)
  x_new = x_old + n_k * s_k
  x_list.append(x_new)
  y_list.append(f(x_new))

  # Time-based Step Size Decay
  n_k = n_k / (1 + t * d)
  t += 1

print('Local minimum occurs at: {:.3f}'.format(x_new))
print("Number of steps:", len(x_list))

# **Gradient descent for Linear Regression**

This notebook recreates basic the gradient descent algorithm introduced in [Understanding Deep Learning Book](https://udlbook.github.io/udlbook/).



In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap

%matplotlib inline


In [ ]:
# Let's create our training data 12 pairs {x_i, y_i}
# We'll try to fit the straight line model to these data
data = np.array([[0.03,0.19,0.34,0.46,0.78,0.81,1.08,1.18,1.39,1.60,1.65,1.90],
                 [0.67,0.85,1.05,1.00,1.40,1.50,1.30,1.54,1.55,1.68,1.73,1.60]])

In [ ]:
# Let's define our model -- just a straight line with intercept phi[0] and slope phi[1]
def model(phi,x):
  y_pred = phi[0]+phi[1] * x
  return y_pred

In [ ]:
# Draw model
def draw_model(data,model,phi,title=None):
  x_model = np.arange(0,2,0.01)
  y_model = model(phi,x_model)

  fix, ax = plt.subplots()
  ax.plot(data[0,:],data[1,:],'bo')
  ax.plot(x_model,y_model,'m-')
  ax.set_xlim([0,2]);ax.set_ylim([0,2])
  ax.set_xlabel('x'); ax.set_ylabel('y')
  ax.set_aspect('equal')
  if title is not None:
    ax.set_title(title)
  plt.show()

In [ ]:
# Initialize the parameters to some arbitrary values and draw the model
phi = np.zeros((2,1))
phi[0] = 0.6      # Intercept
phi[1] = -0.2      # Slope
draw_model(data,model,phi, "Initial parameters")


Now let's compute the sum of squares loss for the training data

In [ ]:
def compute_loss(data_x, data_y, model, phi):
  # First make model predictions from data x
  # Then compute the squared difference between the predictions and true y values
  # Then sum them all and return
  pred_y = model(phi,data_x)
  sq_diff = (pred_y - data_y)**2
  loss = np.sum(sq_diff)

  return loss

In [ ]:
loss = compute_loss(data[0,:],data[1,:],model,np.array([[0.6],[-0.2]]))
print('Calculated loss = %3.3f'%(loss))

Now let's plot the whole loss function

In [ ]:
def draw_loss_function(compute_loss, data,  model, phi_iters = None):
  # Define pretty colormap
  my_colormap_vals_hex =('2a0902', '2b0a03', '2c0b04', '2d0c05', '2e0c06', '2f0d07', '300d08', '310e09', '320f0a', '330f0b', '34100b', '35110c', '36110d', '37120e', '38120f', '39130f', '3a1410', '3b1411', '3c1511', '3d1612', '3e1613', '3f1713', '401714', '411814', '421915', '431915', '451a16', '461b16', '471b17', '481c17', '491d18', '4a1d18', '4b1e19', '4c1f19', '4d1f1a', '4e201b', '50211b', '51211c', '52221c', '53231d', '54231d', '55241e', '56251e', '57261f', '58261f', '592720', '5b2821', '5c2821', '5d2922', '5e2a22', '5f2b23', '602b23', '612c24', '622d25', '632e25', '652e26', '662f26', '673027', '683027', '693128', '6a3229', '6b3329', '6c342a', '6d342a', '6f352b', '70362c', '71372c', '72372d', '73382e', '74392e', '753a2f', '763a2f', '773b30', '783c31', '7a3d31', '7b3e32', '7c3e33', '7d3f33', '7e4034', '7f4134', '804235', '814236', '824336', '834437', '854538', '864638', '874739', '88473a', '89483a', '8a493b', '8b4a3c', '8c4b3c', '8d4c3d', '8e4c3e', '8f4d3f', '904e3f', '924f40', '935041', '945141', '955242', '965343', '975343', '985444', '995545', '9a5646', '9b5746', '9c5847', '9d5948', '9e5a49', '9f5a49', 'a05b4a', 'a15c4b', 'a35d4b', 'a45e4c', 'a55f4d', 'a6604e', 'a7614e', 'a8624f', 'a96350', 'aa6451', 'ab6552', 'ac6552', 'ad6653', 'ae6754', 'af6855', 'b06955', 'b16a56', 'b26b57', 'b36c58', 'b46d59', 'b56e59', 'b66f5a', 'b7705b', 'b8715c', 'b9725d', 'ba735d', 'bb745e', 'bc755f', 'bd7660', 'be7761', 'bf7862', 'c07962', 'c17a63', 'c27b64', 'c27c65', 'c37d66', 'c47e67', 'c57f68', 'c68068', 'c78169', 'c8826a', 'c9836b', 'ca846c', 'cb856d', 'cc866e', 'cd876f', 'ce886f', 'ce8970', 'cf8a71', 'd08b72', 'd18c73', 'd28d74', 'd38e75', 'd48f76', 'd59077', 'd59178', 'd69279', 'd7937a', 'd8957b', 'd9967b', 'da977c', 'da987d', 'db997e', 'dc9a7f', 'dd9b80', 'de9c81', 'de9d82', 'df9e83', 'e09f84', 'e1a185', 'e2a286', 'e2a387', 'e3a488', 'e4a589', 'e5a68a', 'e5a78b', 'e6a88c', 'e7aa8d', 'e7ab8e', 'e8ac8f', 'e9ad90', 'eaae91', 'eaaf92', 'ebb093', 'ecb295', 'ecb396', 'edb497', 'eeb598', 'eeb699', 'efb79a', 'efb99b', 'f0ba9c', 'f1bb9d', 'f1bc9e', 'f2bd9f', 'f2bfa1', 'f3c0a2', 'f3c1a3', 'f4c2a4', 'f5c3a5', 'f5c5a6', 'f6c6a7', 'f6c7a8', 'f7c8aa', 'f7c9ab', 'f8cbac', 'f8ccad', 'f8cdae', 'f9ceb0', 'f9d0b1', 'fad1b2', 'fad2b3', 'fbd3b4', 'fbd5b6', 'fbd6b7', 'fcd7b8', 'fcd8b9', 'fcdaba', 'fddbbc', 'fddcbd', 'fddebe', 'fddfbf', 'fee0c1', 'fee1c2', 'fee3c3', 'fee4c5', 'ffe5c6', 'ffe7c7', 'ffe8c9', 'ffe9ca', 'ffebcb', 'ffeccd', 'ffedce', 'ffefcf', 'fff0d1', 'fff2d2', 'fff3d3', 'fff4d5', 'fff6d6', 'fff7d8', 'fff8d9', 'fffada', 'fffbdc', 'fffcdd', 'fffedf', 'ffffe0')
  my_colormap_vals_dec = np.array([int(element,base=16) for element in my_colormap_vals_hex])
  r = np.floor(my_colormap_vals_dec/(256*256))
  g = np.floor((my_colormap_vals_dec - r *256 *256)/256)
  b = np.floor(my_colormap_vals_dec - r * 256 *256 - g * 256)
  my_colormap = ListedColormap(np.vstack((r,g,b)).transpose()/255.0)

  # Make grid of intercept/slope values to plot
  intercepts_mesh, slopes_mesh = np.meshgrid(np.arange(0.0,2.0,0.02), np.arange(-1.0,1.0,0.002))
  loss_mesh = np.zeros_like(slopes_mesh)
  # Compute loss for every set of parameters
  for idslope, slope in np.ndenumerate(slopes_mesh):
     loss_mesh[idslope] = compute_loss(data[0,:], data[1,:], model, np.array([[intercepts_mesh[idslope]], [slope]]))

  fig,ax = plt.subplots()
  fig.set_size_inches(8,8)
  ax.contourf(intercepts_mesh,slopes_mesh,loss_mesh,256,cmap=my_colormap)
  ax.contour(intercepts_mesh,slopes_mesh,loss_mesh,40,colors=['#80808080'])
  if phi_iters is not None:
    ax.plot(phi_iters[0,:], phi_iters[1,:],'go-')
  ax.set_ylim([1,-1])
  ax.set_xlabel('Intercept $\phi_{0}$'); ax.set_ylabel('Slope, $\phi_{1}$')
  plt.show()

In [ ]:
draw_loss_function(compute_loss, data, model)

Now let's compute the gradient vector for a given set of parameters:

\begin{equation}
\frac{\partial L}{\partial \boldsymbol\phi} = \begin{bmatrix}\frac{\partial L}{\partial \phi_0} \\\frac{\partial L}{\partial \phi_1} \end{bmatrix}.
\end{equation}

In [ ]:
# Expressions for the sum of squares loss and take the
# derivative with respect to phi0 and phi1
def compute_gradient(data_x, data_y, phi):
    
    dl_dphi0 = np.sum(2*(phi[0] + phi[1]*data_x - data_y))
    dl_dphi1 = np.sum(2*data_x*(phi[0] + phi[1]*data_x - data_y))
    
    # Return the gradient
    return np.array([[dl_dphi0],[dl_dphi1]])

We can check we got this right using a trick known as **finite differences**.  If we evaluate the function and then change one of the parameters by a very small amount and normalize by that amount, we get an approximation to the gradient, so:

\begin{align}
\frac{\partial L}{\partial \phi_{0}}&\approx & \frac{L[\phi_0+\delta, \phi_1]-L[\phi_0, \phi_1]}{\delta}\\
\frac{\partial L}{\partial \phi_{1}}&\approx & \frac{L[\phi_0, \phi_1+\delta]-L[\phi_0, \phi_1]}{\delta}
\end{align}

We can't do this when there are many parameters;  for a million parameters, we would have to evaluate the loss function one million plus one times, and usually computing the gradients directly is much more efficient.

In [ ]:
# Compute the gradient using your function
gradient = compute_gradient(data[0,:],data[1,:], phi)
print(f"Computed gradient: ({gradient[0].item():.3f},{gradient[1].item():.3f})")
# Approximate the gradients with finite differences
delta = 0.0001
dl_dphi0_est = (compute_loss(data[0,:],data[1,:],model,phi+np.array([[delta],[0]])) - \
                    compute_loss(data[0,:],data[1,:],model,phi))/delta
dl_dphi1_est = (compute_loss(data[0,:],data[1,:],model,phi+np.array([[0],[delta]])) - \
                    compute_loss(data[0,:],data[1,:],model,phi))/delta
print(f"Approx gradients: ({dl_dphi0_est:.3f},{dl_dphi1_est:.3f})")
# There might be small differences in the last significant figure because finite gradients is an approximation


Now we are ready to perform gradient descent.  We'll use line search plus the helper function loss_function_1D that maps the search along the negative gradient direction in 2D space to a 1D problem (distance along this direction)

In [ ]:
def loss_function_1D(dist_prop, data, model, phi_start, search_direction):
  # Return the loss after moving this far
  return compute_loss(data[0,:], data[1,:], model, phi_start+ search_direction * dist_prop)

def line_search(data, model, phi, gradient, thresh=.00001, max_dist = 0.1, max_iter = 15, verbose=False):
    # Initialize four points along the range we are going to search
    a = 0
    b = 0.33 * max_dist
    c = 0.66 * max_dist
    d = 1.0 * max_dist
    n_iter = 0

    # While we haven't found the minimum closely enough
    while np.abs(b-c) > thresh and n_iter < max_iter:
        # Increment iteration counter (just to prevent an infinite loop)
        n_iter = n_iter+1
        # Calculate all four points
        lossa = loss_function_1D(a, data, model, phi,gradient)
        lossb = loss_function_1D(b, data, model, phi,gradient)
        lossc = loss_function_1D(c, data, model, phi,gradient)
        lossd = loss_function_1D(d, data, model, phi,gradient)

        if verbose:
          print('Iter %d, a=%3.3f, b=%3.3f, c=%3.3f, d=%3.3f'%(n_iter, a,b,c,d))
          print('a %f, b%f, c%f, d%f'%(lossa,lossb,lossc,lossd))

        # Rule #1 If point A is less than points B, C, and D then halve distance from A to points B,C, and D
        if np.argmin((lossa,lossb,lossc,lossd))==0:
          b = a+ (b-a)/2
          c = a+ (c-a)/2
          d = a+ (d-a)/2
          continue;

        # Rule #2 If point b is less than point c then
        #                     point d becomes point c, and
        #                     point b becomes 1/3 between a and new d
        #                     point c becomes 2/3 between a and new d
        if lossb < lossc:
          d = c
          b = a+ (d-a)/3
          c = a+ 2*(d-a)/3
          continue

        # Rule #2 If point c is less than point b then
        #                     point a becomes point b, and
        #                     point b becomes 1/3 between new a and d
        #                     point c becomes 2/3 between new a and d
        a = b
        b = a+ (d-a)/3
        c = a+ 2*(d-a)/3

    # Return average of two middle points
    return (b+c)/2.0

In [ ]:
def gradient_descent_step(phi, data,  model):
  # 1. Compute the gradient (you wrote this function above)
  
  data_x = data[0,:]
  data_y = data[1,:]
    
  gradient = compute_gradient(data_x,data_y, phi)

  print(f"gradient: {gradient} ")

  # 2. Find the best step size alpha using line search function (above) -- use negative gradient as going downhill
  alpha = line_search(data, model, phi, gradient, thresh=.04, max_dist = 2, max_iter = 120, verbose=False)

  print(f"Alpha: {alpha}")  
  
  # 3. Update the parameters phi based on the gradient and the step size alpha.
  phi = phi - alpha* gradient
  print(f"PHI: {phi}")
  return phi

In [ ]:
from IPython.display import clear_output

# Initialize the parameters and draw the model
n_steps = 100
phi_all = np.zeros((2,n_steps+1))
phi_all[0,0] = 1.6
phi_all[1,0] = -0.5

# Measure loss and draw initial model
loss =  compute_loss(data[0,:], data[1,:], model, phi_all[:,0:1])
draw_model(data,model,phi_all[:,0:1], "Initial parameters, Loss = %f"%(loss))

# Repeatedly take gradient descent steps
for c_step in range (n_steps):
  print(f"step:{c_step}")
  # Do gradient descent step
  print(f"current phi_all:{phi_all[:,c_step:c_step+1]}")
  phi_all[:,c_step+1:c_step+2] = gradient_descent_step(phi_all[:,c_step:c_step+1],data, model)
  # Measure loss and draw model
  loss =  compute_loss(data[0,:], data[1,:], model, phi_all[:,c_step+1:c_step+2])
  clear_output(wait=True)
  draw_model(data,model,phi_all[:,c_step+1], "Iteration %d, loss = %f"%(c_step+1,loss))

# Draw the trajectory on the loss function
draw_loss_function(compute_loss, data, model,phi_all)


## Using Gradient Descent to Fit a Linear Regression Model
Let's now consider an example which is a little bit more complicated. 

Consider a simple **linear regression** where we want to see how the temperature affects the noises made by crickets. 

We have a data set of cricket chirp rates at various temperatures. First we'll load that data set in and plot it:

In [ ]:
# Load the dataset
data = np.loadtxt('https://github.com/dtnewman/gradient_descent/raw/master/SGD_data.txt', delimiter=',') #Using "raw" URL from github repository

#print(data)
x = data[:, 0] # Everything from the 0th column
y = data[:, 1] # Everything from the 1st column
m = len(x)     # Number of points

# Plot the data
plot_crickets(data)

Our goal is to find the equation of the straight line $h_\theta(x) = \theta_0 + \theta_1 x$ that best fits our data points. 

The function that we are trying to minimize in this case is:

$J(\theta_0,\theta_1) = {1 \over 2m} \sum\limits_{i=1}^m (h_\theta(x_i)-y_i)^2$

In this case, our gradient will be defined in two dimensions. We must take the derivative with respect to each of the model parameters we wish to estimate, $\theta_0$ and $\theta_1$:

$\frac{\partial}{\partial \theta_0} J(\theta_0,\theta_1) = \frac{1}{m}  \sum\limits_{i=1}^m (h_\theta(x_i)-y_i)$

$\frac{\partial}{\partial \theta_1} J(\theta_0,\theta_1) = \frac{1}{m}  \sum\limits_{i=1}^m ((h_\theta(x_i)-y_i) \cdot x_i)$

Below, we set up our function for $h$, $J$ and the gradient:

In [ ]:
h = lambda theta_0,theta_1,x: theta_0 + theta_1*x

def J(x,y,m,theta_0,theta_1):
    returnValue = 0
    for i in range(m):
        returnValue += (h(theta_0,theta_1,x[i])-y[i])**2
    returnValue = returnValue/(2*m)
    return returnValue

def grad_J(x,y,m,theta_0,theta_1):
    returnValue = np.array([0.,0.])
    for i in range(m):
        returnValue[0] += (h(theta_0,theta_1,x[i])-y[i])
        returnValue[1] += (h(theta_0,theta_1,x[i])-y[i])*x[i]
    returnValue = returnValue/(m)
    return returnValue

And we run our gradient descent algorithm (without adaptive step sizes in this example):

In [ ]:
theta_old = np.array([0.,0.])
theta_new = np.array([1.,1.]) # The algorithm starts at [1,1]
n_k       = 0.001 # step size
precision = 0.001
num_steps = 0
s_k       = float("inf")
delay    = 0.1

while np.linalg.norm(s_k) > precision:
  #""" Toggle to comment out this plotting block (HINT: This converges around 500K...)
  clear_output(wait=True)
  print(f'Iteration {num_steps}\tx: {x_new}\tdiff: {np.linalg.norm(s_k)}')
  plot_crickets(data, theta_new)
  sleep(delay) # To visualize"""

  # Apply Gradient Descent
  num_steps += 1
  theta_old = theta_new
  s_k = -grad_J(x,y,m,theta_old[0],theta_old[1])
  theta_new = theta_old + n_k * s_k

print("Local minimum occurs where:")
print('theta_0 = {:.3f}'.format(theta_new[0]))
print('theta_1 = {:.3f}'.format(theta_new[1]))
print("This took",num_steps,"steps to converge")

For comparison, let's get the actual values for $\theta_0$ and $\theta_1$:

In [ ]:
actual_vals = sp.stats.linregress(x,y)
print(f"Actual values for theta (using built-in linear regression) are:\ntheta_0 = {actual_vals.intercept}\ntheta_1 = {actual_vals.slope}")

So we see that our values are relatively close to the actual values (however our method was incredibly slow). 

If you look at the source code of [linregress](https://github.com/scipy/scipy/blob/master/scipy/stats/_stats_mstats_common.py), it uses the **convariance matrix of x and y** to rapidly compute the next parameter values.

#### Every Single Point is Being Used Each Round!
Notice that in the method above we need to **calculate the gradient in every step** of our algorithm. 

In the example with the crickets, this is not a big deal since there are **only 15 data points**. But imagine that we had 10 million data points! If this were the case, it would certainly make the method above far less efficient.

---

#### (Optional) Applied Homework: Implement Mini-Batch Gradient Descent
In machine learning, the algorithm above is often called <b>batch gradient descent</b> to contrast it with <b>mini-batch gradient descent</b> (which we will not go into here) and <b>stochastic gradient descent</b>.

**Mini-batch gradient descent** is a variation of the gradient descent algorithm that **splits the training dataset into small batches** that are used to calculate model error and update model coefficients.

Implementations may choose to **sum the gradient** over the mini-batch which further reduces the variance of the gradient.

Mini-batch gradient descent seeks to find **a balance between the robustness** of **stochastic gradient descent** and the **efficiency of batch gradient descent**. It is the most common implementation of gradient descent used in the field of deep learning.

---


## Stochastic Gradient Descent (SGD)
As we said above, in batch gradient descent, we must look at every example in the entire training set on every step (in cases where a training set is used for gradient descent). This can be quite slow if the training set is sufficiently large. 

In **stochastic gradient descent**, we update our values after looking at *each* item in the training set, so that we can **start making progress right away**. 

Recall the linear regression example above. In that example, we calculated the gradient for each of the two theta values as follows:

$\frac{\partial}{\partial \theta_0} J(\theta_0,\theta_1) = \frac{1}{m}  \sum\limits_{i=1}^m (h_\theta(x_i)-y_i)$

$\frac{\partial}{\partial \theta_1} J(\theta_0,\theta_1) = \frac{1}{m}  \sum\limits_{i=1}^m ((h_\theta(x_i)-y_i) \cdot x_i)$

Where $h_\theta(x) = \theta_0 + \theta_1 x$

Then we followed this algorithm (where $\alpha$ was a non-adapting stepsize):

&nbsp;&nbsp;&nbsp; 1: &nbsp; Choose initial guess $x_0$ <br>
&nbsp;&nbsp;&nbsp;    2: &nbsp; <b>for</b> k = 0, 1, 2, ... <b>do</b> <br>
&nbsp;&nbsp;&nbsp;    3:   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $s_k$ = -$\nabla f(x_k)$ <br>
&nbsp;&nbsp;&nbsp;    4:   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $x_{k+1} = x_k + \alpha s_k$ <br>
&nbsp;&nbsp;&nbsp;    5: &nbsp;  <b>end for</b>

When the sample data had 15 data points as in the example above, calculating the gradient was not very costly. But for very large data sets, this would not be the case. So instead, we consider a stochastic gradient descent algorithm for simple linear regression such as the following, where m is the size of the data set:

&nbsp;&nbsp;&nbsp; 1: &nbsp; Randomly shuffle the data set <br>
&nbsp;&nbsp;&nbsp;    2: &nbsp; <b>for</b> k = 0, 1, 2, ... <b>do</b> <br>
&nbsp;&nbsp;&nbsp;    3: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <b>for</b> i = 1 to m <b>do</b> <br>
&nbsp;&nbsp;&nbsp;     &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; // Update parameters<br>
&nbsp;&nbsp;&nbsp;    4:   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\begin{bmatrix}
 \theta_{0} \\ 
 \theta_1 \\ 
 \end{bmatrix}=\begin{bmatrix}
 \theta_0 \\ 
 \theta_1 \\ 
 \end{bmatrix}-\alpha\begin{bmatrix}
 2(h_\theta(x_i)-y_i) \\ 
 2x_i(h_\theta(x_i)-y_i) \\ 
 \end{bmatrix}$ <br>
&nbsp;&nbsp;&nbsp;    5: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <b>end for</b> <br> 
&nbsp;&nbsp;&nbsp;    6: &nbsp;  <b>end for</b>

With **batch gradient descent**, we must go through the **entire data set before we make any progress**. With this algorithm though, we can **make progress right away** and continue to make progress as we go through the data set. 

Therefore, **stochastic gradient descent** is often preferred when dealing with **large data sets.**

Typically, with stochastic gradient descent, you will run through the entire data set 1 to 10 times (see value for $k$ in line 2 of the pseudocode above), depending on how fast the data is converging and how large the data set is.

Unlike gradient descent, stochastic gradient descent will tend to **oscillate** <i>near</i> a minimum value rather than continuously getting closer. It **may never actually converge to the minimum** though. 

One way around this is to slowly decrease (**decay**) the step size $\alpha$ as the algorithm runs. However, this is less common than using a fixed $\alpha$.

Let's look at another example where we illustrate the use of stochastic gradient descent for linear regression. In the example below, we'll create a set of 500,000 points around the line $y = 2x+17+\epsilon$, for values of x between 0 and 100:

In [ ]:
f = lambda x: x*2+17+np.random.randn(len(x))*10

x = np.random.random(500000)*100
y = f(x) 
m = len(y)

plot_generic(np.column_stack((x,y)))

First, let's randomly shuffle around our dataset. Note that in this example, this step isn't strictly necessary since the data is already in a random order. However, that obviously may not always be the case:

In [ ]:
from random import shuffle

x_shuf = []
y_shuf = []
index_shuf = list(range(len(x)))
shuffle(index_shuf)
for i in index_shuf:
    x_shuf.append(x[i])
    y_shuf.append(y[i])

Now we'll setup our h function and our cost function, which we will use to check how the value is improving.

In [ ]:
h = lambda theta_0,theta_1,x: theta_0 + theta_1*x

# Function for calculating COST: The cost function is calculated as an average of loss functions
cost = lambda theta_0,theta_1, x_i, y_i: 0.5*(h(theta_0,theta_1,x_i)-y_i)**2

Now we'll run our stochastic gradient descent algorithm. 

To see it's progress, we'll take a loss measurement at every step. 

Every 10,000 steps, we'll get an average cost from the loss measured during the last 10,000 steps and then append that to our cost_list variable. We will run through the entire list 10 times here:

In [ ]:
theta_old = np.array([0.,0.])
theta_new = np.array([1.,1.]) # The algorithm starts at [1,1]
n_k = 0.000005 # step size

iter_num = 0
s_k = np.array([float("inf"),float("inf")])
sum_cost = 0
cost_list = []

# Num. Plot for faster visualization
num_plot = 1000

for j in range(10):
    for i in range(m):
        iter_num += 1
        theta_old = theta_new
        s_k[0] = (h(theta_old[0],theta_old[1],x[i])-y[i])
        s_k[1] = (h(theta_old[0],theta_old[1],x[i])-y[i])*x[i]
        s_k = (-1)*s_k
        theta_new = theta_old + n_k * s_k
        sum_cost += cost(theta_old[0],theta_old[1],x[i],y[i])
        if (i+1) % 10000 == 0:
            print(f'NOTE: Only plotting a subset of {num_plot} points...')
            clear_output(wait=True)
            plot_generic(np.column_stack((x[0:num_plot],y[0:num_plot])), theta_new)
            print(f'Iteration {j}\tData Index: {i+1}\nCost: {sum_cost/10000.0}')
            cost_list.append(sum_cost/10000.0)
            sum_cost = 0   
            
print("\nLocal minimum occurs where:")
print('theta_0 = {:.3f}'.format(theta_new[0]))
print('theta_1 = {:.3f}'.format(theta_new[1]))

As you can see, our values for $\theta_0$ and $\theta_1$ are close to their true values of 17 and 2.

Now, we plot our cost versus the number of iterations. As you can see, the cost goes down quickly at first, but starts to level off as we go through more iterations:

In [ ]:
iterations = np.arange(len(cost_list))*10000
plt.plot(iterations,cost_list)
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.title(f'Model Cost after {len(cost_list)*10000} Iterations')
plt.show()

**Question:** Why do we see 10 regular "peaks" in the curve above? *(Hint: What did we fail to do at the start of each epoch?)*

In [ ]:
# The final model plotted over all the datapoints
plot_generic(np.column_stack((x,y)), theta_new)

 
 ---

 ## Take Away Messages

* **Gradient descent**, also known as <b>steepest descent</b>, is an optimization algorithm for finding the local minimum of a function. 
* Functions must be **differentiable**.
* <b>Gradient ascent</b> is the same as gradient descent, except that it steps in the direction of the **positive of the gradient** and therefore finds **local maximums** instead of minimums. 
* **Learning rate**) can either be **constant** or **decay** according to a schedule.
* Learning Rate Issues: If the **step size is too small**, then convergence will be **very slow**, but if it is **too large**, then the method may **fail to converge at all**. 
* **Mini-batch gradient descent** is a variation of the gradient descent algorithm that **splits the training dataset into small batches** that are used to calculate model error and update model coefficients.
* Mini-batch gradient descent seeks to find **a balance between the robustness** of **stochastic gradient descent** and the **efficiency of batch gradient descent**. It is the most common implementation of gradient descent used in the field of deep learning.
* With **batch gradient descent**, we must go through the **entire data set before we make any progress**.
* In **stochastic gradient descent**, we update our values after looking at *each* item in the training set, so that we can **start making progress right away**. 
* Therefore, **stochastic gradient descent** is often preferred when dealing with **large data sets.**
* Unlike gradient descent, stochastic gradient descent will tend to **oscillate** <i>near</i> a minimum value rather than continuously getting closer. It **may never actually converge to the minimum** though. 

